In [30]:
import numpy as np
import pandas as pd
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

datafame = pd.read_csv('/spambase/spambase.data')
data = np.array(datafame)
X = data[:, 1:len(data)]
y = data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.7)
print(X_train.T.shape)


(57, 3220)
(58, 3220)


In [32]:
Z = np.vstack((X_train.T, y_train))

C = 200
lam = 1./C

def cost(w):
    u = w.T.dot(Z) # as in (23) 
    return (np.sum(np.maximum(0, 1 - u)) + .5*lam*np.sum(w*w)) - .5*lam*w[-1]*w[-1] # no bias 

def grad(w): 
    u = w.T.dot(Z) # as in (23) 
    H = np.where(u < 1)[1] 
    ZS = Z[:, H] 
    g = (-np.sum(ZS, axis = 1, keepdims = True) + lam*w) 
    g[-1] -= lam*w[-1] # no weight decay on bias 
    return g

eps = 1e-6 
def num_grad(w): 
    g = np.zeros_like(w) 
    for i in range(len(w)): 
        wp = w.copy() 
        wm = w.copy() 
        wp[i] += eps 
        wm[i] -= eps  
        g[i] = (cost(wp) - cost(wm))/(2*eps) 
    return g  
 
w0 = np.random.randn(Z.shape[0], 1)  
g1 = grad(w0) 
g2 = num_grad(w0) 
diff = np.linalg.norm(g1 - g2) 
# print('Gradient different: %f' %diff) 

def grad_descent(w0, eta): 
    w = w0 
    it = 0  
    while it < 100000: 
        it = it + 1 
        g = grad(w) 
        w -= eta*g 
        # if (it % 10000) == 1: 
            # print('iter %d' %it + ' cost: %f' %cost(w)) 
        if np.linalg.norm(g) < 1e-5: 
            break  
    return w  
w0 = np.random.randn(Z.shape[0], 1)  
w = grad_descent(w0, 0.001) 
w_hinge = w[:-1].reshape(-1, 1) 
b_hinge = w[-1] 
print(w_hinge.T, b_hinge)

[[0.000e+00 0.000e+00 0.000e+00 ... 9.000e-02 0.000e+00 0.000e+00]
 [0.000e+00 1.900e+00 0.000e+00 ... 9.000e-02 4.200e-01 5.000e-02]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [1.060e+02 1.178e+03 4.000e+00 ... 2.541e+03 1.060e+02 8.960e+02]
 [1.000e+00 1.000e+00 0.000e+00 ... 1.000e+00 0.000e+00 0.000e+00]
 [1.000e+00 1.000e+00 0.000e+00 ... 1.000e+00 0.000e+00 0.000e+00]]
[[-3.84550605e-01  9.95416149e-02 -1.03218020e+00  6.71554464e-01
  -1.71358690e-01  6.73954152e-01 -1.87471865e-01 -5.44724186e-01
   7.42276578e-01  1.67175533e-01  1.52712802e+00  1.03021691e+00
   1.60958082e+00 -5.63237684e-01  5.40019578e-01  2.62945326e-01
  -1.13900393e-01  2.08090142e+00  1.84591989e+00  1.27917445e+00
   8.88586444e-01 -1.17103285e-02  2.12090758e-01  7.75184360e-01
  -3.34350555e-02  1.21400406e+00  2.82009564e-01  4.03606248e-01
   7.00575995e-01 -5.19590968e-01  5.27237337e-01 -8.21956832e-01
  -1.54702128e+00  3.32786899e-01  2.32559128e-01 -3.75915700e-0

In [41]:
print(np.sign(np.dot(w_hinge.T, X_test.T)+ b_hinge))

1380.0
